In [1]:
import transformers
import evaluate
from datasets import load_dataset
from transformers import GenerationConfig, MBart50TokenizerFast
print(transformers.__version__)

/home/arooba/ssd/Shahzaib/.trans_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.36.2


In [2]:
model_checkpoint = "facebook/mbart-large-50"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`Helsinki-NLP/opus-mt-en-ro`](https://huggingface.co/Helsinki-NLP/opus-mt-en-ro) checkpoint. 

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`. We use the English/Romanian part of the WMT dataset here.

In [3]:
raw_datasets = load_dataset("opus_books", "en-fr", split=["train[:50%]"])
metric = evaluate.load("sacrebleu")

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [4]:
raw_datasets = raw_datasets[0]
raw_datasets

Dataset({
    features: ['id', 'translation'],
    num_rows: 63542
})

In [5]:
raw_datasets = raw_datasets.train_test_split(test_size=0.2)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 50833
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 12709
    })
})

To access an actual element, you need to select a split first, then give an index:

In [6]:
raw_datasets["train"][0]

{'id': '22099',
 'translation': {'en': '— Que voulez-vous dire ? »',
  'fr': '"What do you mean?"'}}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [7]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(raw_datasets["train"])

,id,translation
0,45107,"{'en': '""Your name?"" cried his Eminence, for the third time.', 'fr': '-- Votre nom? reprit pour la troisième fois Son Éminence.'}"
1,40123,"{'en': '""Sire, notwithstanding--""', 'fr': '-- Sire, cependant...'}"
2,15666,"{'en': 'I had a belief she loved me even when she left me: that was an atom of sweet in much bitter.', 'fr': 'Même lorsqu'elle m'abandonna, je croyais qu'elle m'aimait encore. C'était ma seule joie au milieu de mes grandes douleurs.'}"
3,7418,"{'en': 'But the spiritless condition which this event threw her into was shortly relieved, and her mind opened again to the agitation of hope, by an article of news which then began to be in circulation. The housekeeper at Netherfield had received orders to prepare for the arrival of her master, who was coming down in a day or two, to shoot there for several weeks.', 'fr': 'Mais la mélancolie ou l’avait plongée cet événement ne résista pas a la nouvelle qui commença bientôt a circuler dans le pays : la femme de charge de Netherfield avait, disait-on, reçu l’ordre de préparer la maison pour l’arrivée prochaine de son maître, qui, a l’occasion de la chasse, venait y passer quelques semaines.'}"
4,61014,"{'en': 'Yes, I am deaf, that is the way I am made.', 'fr': 'Oui, je suis sourd. C’est comme cela que je suis fait.'}"


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [9]:
from transformers import AutoTokenizer
    
tokenizer = MBart50TokenizerFast.from_pretrained(model_checkpoint, src_lang="en_XX", tgt_lang="fr_XX")

For the mBART tokenizer (like we have here), we need to set the source and target languages (so the texts are preprocessed properly). You can check the language codes [here](https://huggingface.co/facebook/mbart-large-cc25) if you are using this notebook on a different pairs of languages.

In [10]:
# if "mbart" in model_checkpoint:
#     tokenizer.src_lang = "en-XX"
#     tokenizer.tgt_lang = "fr-XX"

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

If you are using one of the five T5 checkpoints that require a special prefix to put before the inputs, you should adapt the following cell.

In [11]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to French: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [12]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "fr"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    labels = tokenizer(text_target=targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [13]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[250004, 292, 5813, 88681, 9, 13048, 5365, 705, 340, 2], [250004, 3493, 14373, 759, 18227, 509, 10, 704, 11049, 538, 4989, 1733, 38162, 4, 100, 5941, 21775, 1902, 115081, 8108, 70, 19922, 47, 208877, 13, 25674, 26038, 163, 70, 54591, 3129, 87, 4989, 297, 100, 390, 5155, 136, 48869, 297, 111, 390, 17431, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[250008, 44, 52231, 54, 398, 29459, 2953, 2], [250008, 1775, 150416, 211, 163, 6817, 1003, 143580, 51, 4109, 94176, 674, 4989, 4, 2258, 1806, 224, 13411, 40, 27875, 150086, 9600, 41, 21, 36997, 253, 21, 109453, 3075, 121678, 13, 163, 2301, 19, 45658, 21, 18227, 41, 1647, 25, 90, 11870, 12794, 95, 11992, 82, 13319, 55, 23329, 78453, 21, 56392, 5, 2]]}

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [14]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:  12%|█▏        | 6000/50833 [00:00<00:02, 15101.15 examples/s]

Map: 100%|██████████| 12709/12709 [00:00<00:00, 18171.45 examples/s]


## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [15]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [16]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [17]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]


    # Replace -100 in the preds as well
    # https://github.com/huggingface/transformers/issues/24433 
    preds = np.where(preds!= -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [18]:
my_config = GenerationConfig.from_pretrained(model_checkpoint)
my_config.max_new_tokens = 200

In [19]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
exp = "1"
args = Seq2SeqTrainingArguments(
    output_dir="trained/mBART-50"+"_exp"+exp,
    evaluation_strategy = "steps",
    eval_delay=2000,
    eval_steps=3000,
    save_steps=3000,
    learning_rate=2e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    logging_dir="logs/mBART-50"+"_exp"+exp,
    logging_strategy = "steps",
    logging_steps = 500,
    load_best_model_at_end = True,
    metric_for_best_model ="bleu",
    generation_config=my_config
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [20]:
trainer.model.generation_config

GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 200,
  "max_new_tokens": 200,
  "num_beams": 5,
  "pad_token_id": 1
}

We can now finetune our model by just calling the `train` method:

In [21]:
trainer.train()

  0%|          | 0/63560 [00:00<?, ?it/s]You're using a MBart50TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  1%|          | 500/63560 [02:49<5:36:59,  3.12it/s]

{'loss': 6.0579, 'learning_rate': 0.001984424166142228, 'epoch': 0.16}


  2%|▏         | 1001/63560 [05:39<5:28:40,  3.17it/s]

{'loss': 5.0677, 'learning_rate': 0.0019686910006293267, 'epoch': 0.31}


...

{'loss': 5.0582, 'learning_rate': 0.0019372246696035243, 'epoch': 0.63}


  4%|▍         | 2500/63560 [14:08<5:45:09,  2.95it/s]

{'loss': 5.1052, 'learning_rate': 0.0019214915040906232, 'epoch': 0.79}


  5%|▍         | 3000/63560 [16:57<5:46:29,  2.91it/s]

{'loss': 5.1424, 'learning_rate': 0.0019057583385777219, 'epoch': 0.94}


### Inference

In [ ]:
text = "Legumes share resources with nitrogen-fixing bacteria."

In [ ]:
from transformers import pipeline

translator = pipeline("translation_en_to_fr", model="/home/arooba/ssd/Shahzaib/QuickFormer_Research_Project/machine translation/trained/t5-small/checkpoint-500/")
output = translator(text)

In [ ]:
preds = [item["translation_text"] for item in output]
preds

### Without Pipeline

In [ ]:
from transformers import AutoTokenizer
PATH = "/home/arooba/ssd/Shahzaib/QuickFormer_Research_Project/machine translation/trained/t5-small/checkpoint-500/"
tokenizer = AutoTokenizer.from_pretrained(PATH)
inputs = tokenizer(text, return_tensors="pt", padding=True).input_ids
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(PATH)
outputs = model.generate(inputs, max_new_tokens=120, do_sample=True, top_k=30, top_p=0.95)

In [ ]:
preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
preds

### BLEU Score

In [ ]:
def evaluate_bleu_pipeline(dataset, translator):
    input = [trans["en"] for trans in dataset["translation"]]
    # text_list_en
    target = [trans["fr"] for trans in dataset["translation"]]
    output = translator(input)
    preds = [item["translation_text"] for item in output]
    bleu = evaluate.load("bleu")
    results = bleu.compute(predictions=preds, references=target)
    return results

def evaluate_bleu(dataset, model_path):
    input = [trans["en"] for trans in dataset["translation"]]
    # text_list_en
    target = [trans["fr"] for trans in dataset["translation"]]
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    inputs = tokenizer(input, return_tensors="pt", padding=True).input_ids
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    outputs = model.generate(inputs, max_new_tokens=120, do_sample=True, top_k=30, top_p=0.95)
    preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    bleu = evaluate.load("bleu")
    results = bleu.compute(predictions=preds, references=target)
    return results


In [ ]:
# bleu = evaluate_bleu(raw_datasets["train"][:10], PATH)
from transformers import pipeline
translator = pipeline("translation_en_to_fr", model="/home/arooba/ssd/Shahzaib/QuickFormer_Research_Project/machine translation/trained/t5-small/checkpoint-500/")
bleu = evaluate_bleu_pipeline(raw_datasets["train"][:10], translator)

In [ ]:
print(bleu)